In [1]:
import os
import re

from rdkit import Chem
from openbabel import pybel
import parameter_extractor as pe
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import DrawingOptions

IPythonConsole.molSize = (300, 300)  # Change image size
DrawingOptions.bondLineWidth = 1.8

# Gaussian Input Files Generation
This notebook generates `.gjf` and script files to submit the calculations to gaussian.

This cell creates input files for `SPE_NoRh` structures.

In [2]:
for file in os.listdir("./out/spe"):
    filename = file.split(".")[0]
    ligand_number = re.search(r"\d+[a-z]*", file)[0]
    submit_filename = f"l_{ligand_number}_SPE_NoRh"
    submit_file = f"""#!/bin/csh
#$ -N {submit_filename}
#$ -pe smp 14
#$ -q iqtc08.q
#$ -S /bin/csh
#$ -cwd
#$ -o $HOME/javier/{submit_filename}.log
#$ -e $HOME/javier/{submit_filename}.err
#$ -m e  
#$ -M javier.eusamio@ub.edu 

# Load the modules needed
source /etc/profile.d/modules.csh
module load gaussian/g16b01

cd $TMPDIR
cp -pr $HOME/javier/{submit_filename}.gjf .

# Run the job
g16 < {submit_filename}.gjf > {submit_filename}.out

# Copy the results to our home directory
cp {submit_filename}.out $HOME/javier/"""
    
    # Removes COD + Rh from molecule, converts to .gjf and adds job keywords
    comp = pe.ParameterExtractor(filename, include_no_rh=False)
    editable_mol = Chem.RWMol(comp.mol)
    cod_idx = [a.GetIdx() for a in comp.cod]
    remove_atoms = [a for a in editable_mol.GetAtoms() if a.GetIdx() in cod_idx]
    remove_atoms.append(comp.rh)
    remove_atoms = sorted(remove_atoms, key=lambda x: x.GetIdx(), reverse=True)
    for atom in remove_atoms:
        atom.SetAtomMapNum(atom.GetIdx())
        editable_mol.RemoveAtom(atom.GetIdx())
    pdb_block = Chem.MolToPDBBlock(editable_mol)
    pdb = pybel.readstring("pdb", pdb_block)
    gjf = pdb.write("gjf")
    gjf = re.sub(
        r"!Put Keywords Here, check Charge and Multiplicity.\n#\n\n",
        fr"%nprocshared=14\n%mem=8GB\n# nmr=giao b3lyp/def2TZVP pop=nbo scfcyc=200\n\nLigand {ligand_number} SPE NoRh",
        gjf
    )
    with open(f"./gjf/NoRh/l_{ligand_number}_SPE_NoRh.gjf", "w") as f:
        f.write(gjf)
    with open(f"./gjf/NoRh/{submit_filename}", "w", newline='\n') as f:
        f.write(submit_file)
    print(f"Ligand {ligand_number} done!")

[[' Summary of Natural Population Analysis:                  \n', '                                                          \n', '                                       Natural Population \n', '                Natural  -----------------------------------------------\n', '    Atom  No    Charge         Core      Valence    Rydberg      Total\n', ' -----------------------------------------------------------------------\n', '      P    1    1.02468      9.99682     3.93344    0.04506    13.97532\n', '      P    2    1.06903      9.99669     3.88546    0.04882    13.93097\n', '      O    3   -0.46719      1.99976     6.44701    0.02042     8.46719\n', '      O    4   -0.46138      1.99976     6.44642    0.01521     8.46138\n', '      C    5   -0.24794      1.99943     4.23155    0.01696     6.24794\n', '      C    6   -0.60098      1.99935     4.59385    0.00778     6.60098\n', '      C    7   -0.60676      1.99936     4.59814    0.00926     6.60676\n', '      C    8   -0.60393      1.999